# 意思決定分析と予測の活用｜第2部 決定分析の基礎
https://logics-of-blue.com/decision-analysis-and-forecast-book-support/

## 第4章 期待値に基づく意思決定

### Pythonによる分析の準備

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd

# DataFrameの全角文字の出力をきれいにする
pd.set_option('display.unicode.east_asian_width', True)

In [2]:
# 乱数の種(参考)
np.random.seed(2)

## 期待金額に基づく意思決定

#### 計算方法の確認

In [3]:
# 利得行列
payoff = pd.DataFrame({
    '0台': [-100, -100],
    '1台': [300, 300],
    '2台': [700, -300]
})
payoff.index = ['好況', '不況']
print(payoff)

      0台  1台  2台
好況 -100  300  700
不況 -100  300 -300


In [4]:
# 好況と不況の確率
prob_state = pd.Series([0.4, 0.6])
prob_state.index = ['好況', '不況']
prob_state

好況    0.4
不況    0.6
dtype: float64

In [5]:
# 利得×確率
print(payoff.mul(prob_state, axis=0))

       0台    1台    2台
好況 -40.0  120.0  280.0
不況 -60.0  180.0 -180.0


In [6]:
# 各々の行動をとったときの期待値
emv = payoff.mul(prob_state, axis=0).sum()
emv

0台   -100.0
1台    300.0
2台    100.0
dtype: float64

In [7]:
# 最大値をとるインデックスを取得する。最大値が複数ある場合はすべて出力する。
def argmax_list(series):
    return(list(series[series == series.max()].index))

In [8]:
# 期待値が最大となる行動
argmax_list(emv)

['1台']

#### 効率的な実装

In [9]:
# 期待金額最大化に基づく意思決定を行う関数
def max_emv(probs, payoff_table):
    emv = payoff_table.mul(probs, axis=0).sum()
    max_emv = emv.max()
    a_star = argmax_list(emv)
    return(pd.Series([a_star, max_emv], index=['選択肢', '期待金額']))

In [10]:
max_emv(prob_state, payoff)

選択肢      [1台]
期待金額      300
dtype: object

## 期待値のシミュレーション

In [11]:
# 確率0.4で「700」が、確率0.6で「-300」が出る
print(np.random.choice([700, -300], size=10, p=[0.4, 0.6]))
print(np.random.choice([700, -300], size=10, p=[0.4, 0.6]))

[-300  700 -300 -300 -300  700  700 -300  700  700]
[-300 -300  700 -300  700 -300 -300 -300 -300  700]


In [12]:
# 乱数の種を指定する
np.random.seed(1)
print(np.random.choice([700, -300], size=10, p=[0.4, 0.6]))
np.random.seed(1)
print(np.random.choice([700, -300], size=10, p=[0.4, 0.6]))

[-300 -300  700  700  700  700  700  700  700 -300]
[-300 -300  700  700  700  700  700  700  700 -300]


In [13]:
# 500万個の乱数の生成
np.random.seed(1)
simulation = np.random.choice([700, -300], size=5000000, p=[0.4, 0.6])

In [14]:
# 期待値と平均値
print('期待値:', np.sum(np.array([700, -300]) * np.array([0.4, 0.6])))
print('平均値:', simulation.mean().round(1))

期待値: 100.0
平均値: 100.0
